<a href="https://colab.research.google.com/github/AkshayNagamalla/codeOptimisationGenAI/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install bitsandbytes
!pip install transformers
!pip install torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch
from datetime import datetime

model_path = "hemanth2102/PS-G210"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")

# Context to store only user queries
user_context = []

def process_query(query):
    """
    Process the query by appending it to the user context.
    Generates a new result based on the accumulated context.
    """
    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d")


    # Append the new query and the current date to the user context
    user_context.append(f"{current_date}\nUser: {query}")
    full_context = "\n".join(user_context)

    # Prepare inputs for the model
    messages = [
        {"role": "user", "content": full_context},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Stream output
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    output_tokens = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        max_new_tokens=128000,  # Adjust token length as needed
        use_cache=True,
        temperature=0.8,
        min_p=0.1,
    )

    # Decode the output tokens
    new_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Append only the user's query to the context to avoid repetition
    return new_output

def main():
    print("Welcome to the Llama Model Interactive Query Processor!")
    while True:
        print("\nEnter your query (or type 'exit' to quit):")
        query = input("> ").strip()

        if query.lower() == 'exit':
            print("Exiting the program. Goodbye!")
            break

        # Process the query with accumulated context
        new_output = process_query(query)

        # Display the output
        print(f"\nOutput:\n{new_output}")

if __name__ == "__main__":
    main()


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Welcome to the Llama Model Interactive Query Processor!

Enter your query (or type 'exit' to quit):


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here is a basic HTML code that includes a home page and an about page.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home Page</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <header>
        <nav>
            <ul>
                <li><a href="index.html">Home</a></li>
                <li><a href="about.html">About</a></li>
            </ul>
        </nav>
    </header>
    <main>
        <section id="home">
            <h1>Welcome to Home Page</h1>
            <p>This is the home page of our website.</p>
            <button>Learn More</button>
        </section>
        <section id="about">
            <h1>Welcome to About Page</h1>
            <p>This is the about page of our website.</p>
            <button>Learn More</button>
        </section>
    </main>
    <script src="script.js"></script>
</body>
</html>
```

For the CSS, you can create a

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch
from datetime import datetime

model_path = "hemanth2102/PS-G210"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")

# Context to store only user queries
user_context = []

def convert_code(code_snippet, target_language):
    """
    Convert code to the specified target language.
    """
    target_language = target_language.lower()
    if target_language == "python":
        return f"# Converted to Python:\n{code_snippet}"
    elif target_language == "java":
        return f"// Converted to Java:\npublic class Main {{\n    public static void main(String[] args) {{\n        {code_snippet}\n    }}\n}}"
    elif target_language == "c":
        return f"/* Converted to C */\n#include <stdio.h>\nint main() {{\n    {code_snippet}\n    return 0;\n}}"
    elif target_language == "c++":
        return f"// Converted to C++\n#include <iostream>\nusing namespace std;\nint main() {{\n    {code_snippet}\n    return 0;\n}}"
    else:
        return f"// Conversion to {target_language} not supported."

def process_query(query):
    """
    Process the query by appending it to the user context.
    Generates a new result based on the accumulated context.
    """
    # Check if the query is a conversion request
    if query.startswith("convert:"):
        parts = query.split(":", 1)[1].strip().split("|")
        code_snippet = parts[0].strip()
        target_language = parts[1].strip() if len(parts) > 1 else "Python"
        return convert_code(code_snippet, target_language)

    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Append the new query and the current date to the user context
    user_context.append(f"{current_date}\nUser: {query}")
    full_context = "\n".join(user_context)

    # Prepare inputs for the model
    messages = [
        {"role": "user", "content": full_context},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Stream output
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    output_tokens = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        max_new_tokens=128000,  # Adjust token length as needed
        use_cache=True,
        temperature=0.8,
        min_p=0.1,
    )

    # Decode the output tokens
    new_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Append only the user's query to the context to avoid repetition
    return new_output

def main():
    print("Welcome to the Llama Model Interactive Query Processor!")
    while True:
        print("\nEnter your query (or type 'exit' to quit):")
        query = input("> ").strip()

        if query.lower() == 'exit':
            print("Exiting the program. Goodbye!")
            break

        # Process the query with accumulated context
        new_output = process_query(query)

        # Display the output
        print(f"\nOutput:\n{new_output}")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is a complete Python code for adding two numbers, taking input from the user, and validating the input:

```python
def add_numbers():
    # Take input from the user
    num1 = float(input("Enter the first number: "))
    num2 = float(input("Enter the second number: "))

    # Validate the input
    if num1 == float('inf') or num2 == float('inf'):
        print("Invalid input. Please enter a valid number.")
        return
    elif num1 == float('-inf') or num2 == float('-inf'):
        print("Invalid input. Please enter a valid number.")
        return
    elif num1 == num2:
        print("Invalid input. Please enter two different numbers.")
        return

    # Calculate and print the result
    result = num1 + num2
    print(f"The sum of {num1} and {num2} is {result}")

if __name__ == "__main__":
    add_numbers()
```

In this code, we first take input from the user using the input function. We then validate the input to ensure it is a valid number. If the input is not valid, we

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's how you could implement the addition of two numbers in Java and Python, including taking inputs and using switch cases:

Java:
```java
import java.util.Scanner;

public class Main {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        System.out.println("Enter the first number: ");
        double num1 = scanner.nextDouble();
        System.out.println("Enter the second number: ");
        double num2 = scanner.nextDouble();
        System.out.println("The sum of the two numbers is: " + (num1 + num2));
        scanner.close();
    }
}
```

Python:
```python
def add_numbers():
    num1 = float(input("Enter the first number: "))
    num2 = float(input("Enter the second number: "))
    print(f"The sum of the two numbers is: {num1 + num2}")

add_numbers()
```

If you want to use switch cases in Python, you could use a dictionary to map the cases to the corresponding actions. However, Python does not have a built-in switch case like so

KeyboardInterrupt: Interrupted by user